In [87]:
import numpy as np
import os
import h5py
import pyedflib
from tqdm import tqdm
import pandas as pd
from scipy.signal import butter, filtfilt, iirnotch, sosfilt, sosfiltfilt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score,  precision_recall_fscore_support
from imblearn.over_sampling import SMOTE
import time

First extract the seizure files from the summary.txt

In [33]:
edf_path = "/Users/folasewaabdulsalam/Seizure_Onset/edf_files"
summary_path= "/Users/folasewaabdulsalam/Seizure_Onset/summary_files"
output_path = "/Users/folasewaabdulsalam/Seizure_Onset/seizure_files"
os.makedirs(output_path , exist_ok=True)

def extract_seizure_segments(edf_path, summary_path, output_path):
    # Process each summary file
    for summary_file in os.listdir(summary_path):
        if summary_file.endswith(".txt"):
            participant_id = summary_file.replace("-summary.txt", "")
            
            # Read the summary file
            with open(os.path.join(summary_path, summary_file), "r") as f:
                current_file = None
                seizure_times = {}
                
                for line in f:
                    # Detect file names
                    if line.startswith("File Name:"):
                        current_file = line.split(":")[1].strip().replace(".edf", "")
                        seizure_times[current_file] = []

                    # Detect seizure start and end times
                    if "Seizure Start Time:" in line:
                        start_time = int(line.split(":")[1].strip().replace(" seconds", ""))
                        seizure_times[current_file].append(("start", start_time))

                    if "Seizure End Time:" in line:
                        end_time = int(line.split(":")[1].strip().replace(" seconds", ""))
                        seizure_times[current_file].append(("end", end_time))

                # Extract and save seizure segments
                for file_name, events in seizure_times.items():
                    edf_file = f"{file_name}.edf"
                    edf_file_path = os.path.join(edf_path, edf_file)

                    if os.path.exists(edf_file_path):
                        with pyedflib.EdfReader(edf_file_path) as f:
                            signals = np.array([f.readSignal(i) for i in range(f.signals_in_file)])
                            fs = f.getSampleFrequency(0)

                            # Extract seizure segments
                            starts = [t[1] for t in events if t[0] == "start"]
                            ends = [t[1] for t in events if t[0] == "end"]

                            for start, end in zip(starts, ends):
                                start_sample = int(start * fs)
                                end_sample = int(end * fs)
                                seizure_segment = signals[:, start_sample:end_sample]

                                # Save the seizure segment
                                output_file = os.path.join(output_path, f"{file_name}_{start}_{end}_seizure.npy")
                                np.save(output_file, seizure_segment)
                                print(f"Saved {output_file}")

print("Seizure extraction complete!")


# Run the extraction
extract_seizure_segments(edf_path, summary_path, output_path)


Seizure extraction complete!
Saved /Users/folasewaabdulsalam/Seizure_Onset/seizure_files/chb03_01_362_414_seizure.npy
Saved /Users/folasewaabdulsalam/Seizure_Onset/seizure_files/chb03_02_731_796_seizure.npy
Saved /Users/folasewaabdulsalam/Seizure_Onset/seizure_files/chb03_03_432_501_seizure.npy
Saved /Users/folasewaabdulsalam/Seizure_Onset/seizure_files/chb03_04_2162_2214_seizure.npy
Saved /Users/folasewaabdulsalam/Seizure_Onset/seizure_files/chb03_34_1982_2029_seizure.npy
Saved /Users/folasewaabdulsalam/Seizure_Onset/seizure_files/chb03_35_2592_2656_seizure.npy
Saved /Users/folasewaabdulsalam/Seizure_Onset/seizure_files/chb03_36_1725_1778_seizure.npy
Saved /Users/folasewaabdulsalam/Seizure_Onset/seizure_files/chb24_01_480_505_seizure.npy
Saved /Users/folasewaabdulsalam/Seizure_Onset/seizure_files/chb24_01_2451_2476_seizure.npy
Saved /Users/folasewaabdulsalam/Seizure_Onset/seizure_files/chb24_03_231_260_seizure.npy
Saved /Users/folasewaabdulsalam/Seizure_Onset/seizure_files/chb24_03_28

Convert to .h5

In [34]:
#convert full files to .h5
edf_path = "/Users/folasewaabdulsalam/Seizure_Onset/edf_files"
h5_path= "/Users/folasewaabdulsalam/Seizure_Onset/h5_files"
os.makedirs(h5_path, exist_ok=True)

def convert_edf_to_h5(edf_path, h5_path):
    for edf_file in os.listdir(edf_path):
        if edf_file.endswith(".edf"):
            edf_file_path = os.path.join(edf_path, edf_file)
            h5_file_path = os.path.join(h5_path, edf_file.replace(".edf", ".h5"))
            
            with pyedflib.EdfReader(edf_file_path) as f:
                signals = np.array([f.readSignal(i) for i in range(f.signals_in_file)])
                channels = f.getSignalLabels()
                sample_rate = f.getSampleFrequency(0)

            # Save as .h5
            with h5py.File(h5_file_path, "w") as hf:
                hf.create_dataset("data", data=signals, compression="gzip")
                hf.attrs["channels"] = channels
                hf.attrs["sample_rate"] = sample_rate
            
            print(f"Converted {edf_file} to {h5_file_path}")

print("Converting full .edf files...")
convert_edf_to_h5(edf_path, h5_path)


Converting full .edf files...
Converted chb24_03.edf to /Users/folasewaabdulsalam/Seizure_Onset/h5_files/chb24_03.h5
Converted chb24_17.edf to /Users/folasewaabdulsalam/Seizure_Onset/h5_files/chb24_17.h5
Converted chb24_16.edf to /Users/folasewaabdulsalam/Seizure_Onset/h5_files/chb24_16.h5
Converted chb24_02.edf to /Users/folasewaabdulsalam/Seizure_Onset/h5_files/chb24_02.h5
Converted chb01_43.edf to /Users/folasewaabdulsalam/Seizure_Onset/h5_files/chb01_43.h5
Converted chb24_14.edf to /Users/folasewaabdulsalam/Seizure_Onset/h5_files/chb24_14.h5
Converted chb23_09.edf to /Users/folasewaabdulsalam/Seizure_Onset/h5_files/chb23_09.h5
Converted chb23_20.edf to /Users/folasewaabdulsalam/Seizure_Onset/h5_files/chb23_20.h5
Converted chb24_01.edf to /Users/folasewaabdulsalam/Seizure_Onset/h5_files/chb24_01.h5
Converted chb23_08.edf to /Users/folasewaabdulsalam/Seizure_Onset/h5_files/chb23_08.h5
Converted chb24_15.edf to /Users/folasewaabdulsalam/Seizure_Onset/h5_files/chb24_15.h5
Converted chb

***startttttt****

Preprocessing seziures and non seizures

In [44]:
#extracting seizure annotations from summary.txt
summary_path = "/Users/folasewaabdulsalam/Seizure_Onset/summary_files"
annotations_file = "/Users/folasewaabdulsalam/Seizure_Onset/seizure_annotations.txt"

def extract_seizure_annotations(summary_path, output_file):
    seizure_annotations = {}

    for summary_file in os.listdir(summary_path):
        if summary_file.endswith(".txt"):
            with open(os.path.join(summary_path, summary_file), "r") as f:
                current_file = None
                for line in f:
                    # Detect file names
                    if line.startswith("File Name:"):
                        current_file = line.split(":")[1].strip().replace(".edf", "")
                        seizure_annotations[current_file] = []

                    # Detect seizure start and end times
                    if "Seizure Start Time:" in line:
                        start_time = int(line.split(":")[1].strip().replace(" seconds", ""))
                        seizure_annotations[current_file].append(("start", start_time))

                    if "Seizure End Time:" in line:
                        end_time = int(line.split(":")[1].strip().replace(" seconds", ""))
                        seizure_annotations[current_file].append(("end", end_time))

    # Save the annotations as a single file
    with open(output_file, "w") as f:
        for file, events in seizure_annotations.items():
            for event in events:
                f.write(f"{file} {event[0]} {event[1]}\n")
    
    print(f"Annotations saved to {output_file}")

# Run the extraction
extract_seizure_annotations(summary_path, annotations_file)

Annotations saved to /Users/folasewaabdulsalam/Seizure_Onset/seizure_annotations.txt


In [71]:

# Directories
h5_full_path = "/Users/folasewaabdulsalam/Seizure_Onset/h5_files"
annotations_file = "/Users/folasewaabdulsalam/Seizure_Onset/seizure_annotations.txt"
preprocessed_path = "/Users/folasewaabdulsalam/Seizure_Onset/preprocessed_data"
os.makedirs(preprocessed_path, exist_ok=True)

FILTER_BANDS = [
    (0.5, 3.625),
    (3.625, 6.75),
    (6.75, 9.875),
    (9.875, 13.0),
    (13.0, 16.125),
    (16.125, 19.25),
    (19.25, 22.375),
    (22.375, 25.0)
]

SELECTED_CHANNELS = list(range(18))

WINDOW_SIZE = 2 * 256  # 2 seconds at 256 Hz
FS = 256  # Sampling rate

def load_annotations(annotations_file):
    """
    Load seizure annotations from a plain text file.
    """
    seizure_annotations = {}
    with open(annotations_file, "r") as f:
        for line in f:
            # Split by space
            parts = line.strip().split()
            
            # Extract file name (without the _preprocessed suffix)
            file_name = parts[0].strip()
            
            # Extract event and time
            event = parts[1].strip().encode("utf-8")
            time = int(parts[2].strip())
            
            # Store the annotation
            if file_name not in seizure_annotations:
                seizure_annotations[file_name] = []
            seizure_annotations[file_name].append((event, time))
    
    # Convert each list to a structured numpy array
    for key in seizure_annotations.keys():
        seizure_annotations[key] = np.array(seizure_annotations[key], dtype=[("event", "S5"), ("time", "i4")])
    
    return seizure_annotations

def preprocess_h5_files(h5_full_path, preprocessed_path, annotations_file):
    """
    Preprocess .h5 files by applying bandpass filtering and channel selection.
    """
    # Load annotations
    seizure_annotations = load_annotations(annotations_file)

    for file_name in tqdm(os.listdir(h5_full_path), desc="Preprocessing .h5 Files"):
        if file_name.endswith(".h5"):
            file_path = os.path.join(h5_full_path, file_name)
            output_file_path = os.path.join(preprocessed_path, file_name.replace(".h5", "_preprocessed.h5"))

            with h5py.File(file_path, "r") as hf:
                signals = hf["data"][:]

                # Select only the first 18 channels
                if signals.shape[0] < 18:
                    print(f"⚠️ WARNING: Skipping file {file_name} with fewer than 18 channels. Found {signals.shape[0]}")
                    continue

                signals = signals[:18, :]  # Use the first 18 channels

                # Apply bandpass filtering
                sos = butter(4, [0.5, 25.0], btype='band', fs=FS, output='sos')
                filtered_signals = sosfiltfilt(sos, signals, axis=1)

                # Save the preprocessed data
                with h5py.File(output_file_path, "w") as out_hf:
                    out_hf.create_dataset("data", data=filtered_signals, compression="gzip")
                    out_hf.attrs["channels"] = list(range(18))

                    # Attach annotations if available
                    file_base_name = file_name.replace(".h5", "")
                    if file_base_name in seizure_annotations:
                        out_hf.attrs["seizure_annotations"] = seizure_annotations[file_base_name]

                print(f"Saved {output_file_path}")

preprocess_h5_files(h5_full_path, preprocessed_path, annotations_file)
print("\nPreprocessing complete!")



Preprocessing .h5 Files:   1%|▏         | 1/73 [00:04<05:40,  4.73s/it]

Saved /Users/folasewaabdulsalam/Seizure_Onset/preprocessed_data/chb05_36_preprocessed.h5


Preprocessing .h5 Files:   3%|▎         | 2/73 [00:14<09:14,  7.81s/it]

Saved /Users/folasewaabdulsalam/Seizure_Onset/preprocessed_data/chb23_06_preprocessed.h5


Preprocessing .h5 Files:   4%|▍         | 3/73 [00:34<15:15, 13.08s/it]

...

Saved /Users/folasewaabdulsalam/Seizure_Onset/preprocessed_data/chb05_01_preprocessed.h5


Preprocessing .h5 Files:  99%|█████████▊| 72/73 [06:29<00:04,  4.50s/it]

Saved /Users/folasewaabdulsalam/Seizure_Onset/preprocessed_data/chb03_34_preprocessed.h5


Preprocessing .h5 Files: 100%|██████████| 73/73 [06:33<00:00,  5.39s/it]

Saved /Users/folasewaabdulsalam/Seizure_Onset/preprocessed_data/chb01_04_preprocessed.h5

Preprocessing complete!


In [81]:
#extracting features
feature_path = "/Users/folasewaabdulsalam/Seizure_Onset/feature_path"
os.makedirs(feature_path, exist_ok=True)

def extract_spectral_energy(data, fs=FS):
    """
    Extracts spectral energy features for each 2-second window.
    """
    num_channels = data.shape[0]
    num_samples = data.shape[1]
    num_windows = (num_samples - WINDOW_SIZE) // WINDOW_SIZE + 1

    feature_vectors = []

    for i in range(num_windows):
        start = i * WINDOW_SIZE
        end = start + WINDOW_SIZE
        window = data[:, start:end]

        # Extract 8-band energy per channel
        features = []
        for low_cut, high_cut in FILTER_BANDS:
            nyquist = 0.5 * fs
            low = low_cut / nyquist
            high = high_cut / nyquist
            b, a = butter(4, [low, high], btype='band')
            filtered = filtfilt(b, a, window, axis=1)
            
            # Calculate energy (sum of squares)
            energy = np.sum(filtered ** 2, axis=1)  # Shape: (18,)
            features.append(energy)

        # Flatten to create the final feature vector (8 bands × 18 channels = 144)
        feature_vector = np.concatenate(features)
        feature_vectors.append(feature_vector)

    return np.array(feature_vectors)

def extract_and_save_features(preprocessed_path, feature_path, annotations_file):
    """
    Extract features from all preprocessed files.
    """
    # Load annotations
    seizure_annotations = load_annotations(annotations_file)

    # Get all preprocessed .h5 files
    h5_files = [f for f in os.listdir(preprocessed_path) if f.endswith("_preprocessed.h5")]
    
    for h5_file in tqdm(h5_files, desc="Extracting Features"):
        file_path = os.path.join(preprocessed_path, h5_file)
        output_file = os.path.join(feature_path, h5_file.replace("_preprocessed.h5", "_features.h5"))
        file_base_name = h5_file.replace("_preprocessed.h5", "")

        # Load the preprocessed data
        with h5py.File(file_path, "r") as hf:
            signals = hf["data"][:]
            # Use only the first 18 channels
            signals = signals[SELECTED_CHANNELS, :]
            features = extract_spectral_energy(signals)
        
        # Extract and add annotations
        labels = np.zeros(features.shape[0], dtype=int)  # Default to non-seizure
        if file_base_name in seizure_annotations:
            for event, time in seizure_annotations[file_base_name]:
                if event == b"start":
                    start_idx = time // 2
                elif event == b"end":
                    end_idx = time // 2
                    labels[start_idx:end_idx] = 1  # Mark seizure windows
        
        # Save the features and labels
        with h5py.File(output_file, "w") as out_hf:
            out_hf.create_dataset("features", data=features, compression="gzip")
            out_hf.create_dataset("labels", data=labels, compression="gzip")
            out_hf.attrs["channels"] = SELECTED_CHANNELS
            if file_base_name in seizure_annotations:
                out_hf.attrs["seizure_annotations"] = seizure_annotations[file_base_name]
        
        print(f"Saved {output_file}")

# Run the feature extraction
extract_and_save_features(preprocessed_path, feature_path, annotations_file)

print("\nFeature extraction complete!")

Extracting Features:   1%|▏         | 1/73 [00:05<06:08,  5.12s/it]

Saved /Users/folasewaabdulsalam/Seizure_Onset/feature_path/chb01_46_features.h5


Extracting Features:   3%|▎         | 2/73 [00:09<05:52,  4.97s/it]

Saved /Users/folasewaabdulsalam/Seizure_Onset/feature_path/chb01_01_features.h5


Extracting Features:   4%|▍         | 3/73 [00:15<05:55,  5.08s/it]

...

Saved /Users/folasewaabdulsalam/Seizure_Onset/feature_path/chb01_43_features.h5


Extracting Features:  99%|█████████▊| 72/73 [07:08<00:05,  5.68s/it]

Saved /Users/folasewaabdulsalam/Seizure_Onset/feature_path/chb02_27_features.h5


Extracting Features: 100%|██████████| 73/73 [07:18<00:00,  6.00s/it]

Saved /Users/folasewaabdulsalam/Seizure_Onset/feature_path/chb23_06_features.h5

Feature extraction complete!


In [86]:
#loading features and labels with the annotations
from collections import Counter
def load_features_and_labels(feature_path, annotations_file):
    """
    Load features and labels from all feature files.
    """
    features = []
    labels = []

    # Load annotations
    seizure_annotations = {}
    with open(annotations_file, "r") as f:
        for line in f:
            parts = line.strip().split()
            file_name = parts[0].strip()
            event_type = parts[1].strip().encode("utf-8")
            event_time = int(parts[2].strip())
            
            if file_name not in seizure_annotations:
                seizure_annotations[file_name] = []
            seizure_annotations[file_name].append((event_type, event_time))

    for file_name in os.listdir(feature_path):
        if file_name.endswith("_features.h5"):
            file_base_name = file_name.replace("_features.h5", "")
            file_path = os.path.join(feature_path, file_name)
            
            with h5py.File(file_path, "r") as hf:
                data = hf["features"][:]
                
                # Check for seizure annotations
                file_annotations = seizure_annotations.get(file_base_name, [])
                labels_for_file = np.zeros(data.shape[0], dtype=int)

                # Mark seizure windows
                for event, time in file_annotations:
                    if event == b"start":
                        start_idx = time // 2
                        labels_for_file[start_idx:] = 1
                    elif event == b"end":
                        end_idx = time // 2
                        labels_for_file[end_idx:] = 0

                features.append(data)
                labels.append(labels_for_file)

    # Convert to numpy arrays
    features = np.vstack(features)
    labels = np.concatenate(labels)
    
    return features, labels



In [93]:

#training the model
model_results_path = "/Users/folasewaabdulsalam/Seizure_Onset/model_results"

# Load Data
features, labels = load_features_and_labels(feature_path, annotations_file)

# Preserve Original Seizure-to-Non-Seizure Ratio
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, stratify=labels, random_state=42
)

# Print Ratio
print("\nTraining Set Ratio:", Counter(y_train))
print("Test Set Ratio:", Counter(y_test))

# Normalize Features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Balance Training Data
smote = SMOTE(random_state=42, k_neighbors=5)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Print Balanced Ratio
print("\nBalanced Training Set Ratio:", Counter(y_train_balanced))

# Define Models
models = {
    "Naive Bayes": GaussianNB(),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
}

# Train and Evaluate
for model_name, model in models.items():
    print(f"\nTraining {model_name}...")
    model.fit(X_train_balanced, y_train_balanced)
    y_pred = model.predict(X_test)

    # Metrics
    f1 = f1_score(y_test, y_pred, average="weighted")
    precision, recall, f1_scores, _ = precision_recall_fscore_support(y_test, y_pred, average="binary")
    cm = confusion_matrix(y_test, y_pred)

    # Sensitivity and Specificity
    tn, fp, fn, tp = cm.ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    # Latency Calculation (Onset Delay)
    latency = np.mean([max(0, i - j) for i, j in zip(np.where(y_pred == 1)[0], np.where(y_test == 1)[0])]) if np.any(y_test == 1) else 0

    # Save Results
    with open(os.path.join(model_results_path, f"{model_name}_results.txt"), "w") as f:
        f.write(f"Model: {model_name}\n")
        f.write(f"F1 Score: {f1:.4f}\n")
        f.write(f"Sensitivity (Recall): {sensitivity:.4f}\n")
        f.write(f"Specificity: {specificity:.4f}\n")
        f.write(f"Latency: {latency:.4f} samples\n")
        f.write("Confusion Matrix:\n")
        f.write(f"{cm}\n")
        f.write("Classification Report:\n")
        f.write(classification_report(y_test, y_pred))

    print(f"\nModel: {model_name}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Sensitivity (Recall): {sensitivity:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print(f"Latency: {latency:.4f} samples")
    print("Confusion Matrix:")
    print(cm)
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

print("\nTraining Complete!")


Training Set Ratio: Counter({np.int64(0): 127028, np.int64(1): 798})
Test Set Ratio: Counter({np.int64(0): 31757, np.int64(1): 200})


/Users/folasewaabdulsalam/Seizure_Onset/venv/lib/python3.9/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Balanced Training Set Ratio: Counter({np.int64(0): 127028, np.int64(1): 127028})

Training Naive Bayes...

Model: Naive Bayes
F1 Score: 0.9814
Sensitivity (Recall): 0.4800
Specificity: 0.9764
Latency: 0.4000 samples
Confusion Matrix:
[[31008   749]
 [  104    96]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     31757
           1       0.11      0.48      0.18       200

    accuracy                           0.97     31957
   macro avg       0.56      0.73      0.59     31957
weighted avg       0.99      0.97      0.98     31957


Training Random Forest...

Model: Random Forest
F1 Score: 0.9970
Sensitivity (Recall): 0.7200
Specificity: 0.9988
Latency: 1359.5519 samples
Confusion Matrix:
[[31718    39]
 [   56   144]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     31757
           1       0.79      0.72      0.75       200

    ac

/Users/folasewaabdulsalam/Seizure_Onset/venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [03:48:21] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



Model: XGBoost
F1 Score: 0.9975
Sensitivity (Recall): 0.8600
Specificity: 0.9983
Latency: 4.1150 samples
Confusion Matrix:
[[31703    54]
 [   28   172]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     31757
           1       0.76      0.86      0.81       200

    accuracy                           1.00     31957
   macro avg       0.88      0.93      0.90     31957
weighted avg       1.00      1.00      1.00     31957


Training Complete!


In [94]:
#training 2
# Paths
model_results_path = "/Users/folasewaabdulsalam/Seizure_Onset/model_results"
os.makedirs(model_results_path, exist_ok=True)

# Load Features and Labels
features, labels = load_features_and_labels(feature_path, annotations_file)

# Preserve Original Seizure-to-Non-Seizure Ratio
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, stratify=labels, random_state=42
)

# Print Ratio
print("\nTraining Set Ratio:", Counter(y_train))
print("Test Set Ratio:", Counter(y_test))

# Normalize Features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Balance Training Data
smote = SMOTE(random_state=42, k_neighbors=5)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Print Balanced Ratio
print("\nBalanced Training Set Ratio:", Counter(y_train_balanced))

# Define Models
models = {
    "Naive Bayes": GaussianNB(),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
}

# Train and Evaluate
for model_name, model in models.items():
    print(f"\nTraining {model_name}...")
    model.fit(X_train_balanced, y_train_balanced)
    y_pred = model.predict(X_test)

    # Metrics
    f1 = f1_score(y_test, y_pred, average="weighted")
    precision, recall, f1_scores, _ = precision_recall_fscore_support(y_test, y_pred, average="binary")
    cm = confusion_matrix(y_test, y_pred)

    # Sensitivity and Specificity
    tn, fp, fn, tp = cm.ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    # Improved Latency Calculation (Onset Delay in Seconds)
    latency = 0
    true_onsets = np.where(y_test == 1)[0]
    predicted_onsets = np.where(y_pred == 1)[0]

    if len(true_onsets) > 0 and len(predicted_onsets) > 0:
        latency_list = []
        for pred in predicted_onsets:
            # Find the closest true onset before the predicted onset
            closest_true = true_onsets[true_onsets <= pred]
            if len(closest_true) > 0:
                latency_list.append((pred - closest_true[-1]) * 2)  # Convert to seconds

        # Calculate the average latency if any valid matches were found
        latency = np.mean(latency_list) if latency_list else 0

    # Save Results
    with open(os.path.join(model_results_path, f"{model_name}_results2.txt"), "w") as f:
        f.write(f"Model: {model_name}\n")
        f.write(f"F1 Score: {f1:.4f}\n")
        f.write(f"Sensitivity (Recall): {sensitivity:.4f}\n")
        f.write(f"Specificity: {specificity:.4f}\n")
        f.write(f"Latency: {latency:.4f} seconds\n")
        f.write("Confusion Matrix:\n")
        f.write(f"{cm}\n")
        f.write("Classification Report:\n")
        f.write(classification_report(y_test, y_pred))

    print(f"\nModel: {model_name}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Sensitivity (Recall): {sensitivity:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print(f"Latency: {latency:.4f} seconds")
    print("Confusion Matrix:")
    print(cm)
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

print("\nTraining Complete!")



Training Set Ratio: Counter({np.int64(0): 127028, np.int64(1): 798})
Test Set Ratio: Counter({np.int64(0): 31757, np.int64(1): 200})


/Users/folasewaabdulsalam/Seizure_Onset/venv/lib/python3.9/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Balanced Training Set Ratio: Counter({np.int64(0): 127028, np.int64(1): 127028})

Training Naive Bayes...

Model: Naive Bayes
F1 Score: 0.9814
Sensitivity (Recall): 0.4800
Specificity: 0.9764
Latency: 282.6888 seconds
Confusion Matrix:
[[31008   749]
 [  104    96]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     31757
           1       0.11      0.48      0.18       200

    accuracy                           0.97     31957
   macro avg       0.56      0.73      0.59     31957
weighted avg       0.99      0.97      0.98     31957


Training Random Forest...


KeyboardInterrupt: 

In [97]:
#trainig 3
# Load Features and Labels
features, labels = load_features_and_labels(feature_path, annotations_file)

# Preserve Original Seizure-to-Non-Seizure Ratio
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, stratify=labels, random_state=42
)

# Print Initial Ratio
print("\nTraining Set Ratio:", Counter(y_train))
print("Test Set Ratio:", Counter(y_test))

# Normalize Features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Downsample the Majority Class
def downsample(X, y):
    """
    Downsamples the majority class to match the size of the minority class.
    """
    # Separate the majority and minority classes
    X_majority = X[y == 0]
    y_majority = y[y == 0]
    X_minority = X[y == 1]
    y_minority = y[y == 1]

    # Downsample the majority class
    np.random.seed(42)
    majority_indices = np.random.choice(len(X_majority), len(X_minority), replace=False)
    X_majority_downsampled = X_majority[majority_indices]
    y_majority_downsampled = y_majority[majority_indices]

    # Combine the downsampled majority class with the minority class
    X_balanced = np.vstack((X_majority_downsampled, X_minority))
    y_balanced = np.hstack((y_majority_downsampled, y_minority))

    # Shuffle the balanced data
    indices = np.arange(len(y_balanced))
    np.random.shuffle(indices)
    return X_balanced[indices], y_balanced[indices]

X_train_balanced, y_train_balanced = downsample(X_train, y_train)

# Print Balanced Ratio
print("\nBalanced Training Set Ratio:", Counter(y_train_balanced))

# Define Models
models = {
    "Naive Bayes": GaussianNB(),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
}

# Train and Evaluate
for model_name, model in models.items():
    print(f"\nTraining {model_name}...")
    model.fit(X_train_balanced, y_train_balanced)
    y_pred = model.predict(X_test)

    # Metrics
    f1 = f1_score(y_test, y_pred, average="weighted")
    precision, recall, f1_scores, _ = precision_recall_fscore_support(y_test, y_pred, average="binary")
    cm = confusion_matrix(y_test, y_pred)

    # Sensitivity and Specificity
    tn, fp, fn, tp = cm.ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    # Improved Latency Calculation (Onset Delay in Seconds)
    latency = 0
    true_onsets = np.where(y_test == 1)[0]
    predicted_onsets = np.where(y_pred == 1)[0]

    if len(true_onsets) > 0 and len(predicted_onsets) > 0:
        latency_list = []
        for pred in predicted_onsets:
            # Find the closest true onset before the predicted onset
            closest_true = true_onsets[true_onsets <= pred]
            if len(closest_true) > 0:
                latency_list.append((pred - closest_true[-1]) * 2)  # Convert to seconds

        # Calculate the average latency if any valid matches were found
        latency = np.mean(latency_list) if latency_list else 0

    # Save Results
    with open(os.path.join(model_results_path, f"{model_name}_results3.txt"), "w") as f:
        f.write(f"Model: {model_name}\n")
        f.write(f"F1 Score: {f1:.4f}\n")
        f.write(f"Sensitivity (Recall): {sensitivity:.4f}\n")
        f.write(f"Specificity: {specificity:.4f}\n")
        f.write(f"Latency: {latency:.4f} seconds\n")
        f.write("Confusion Matrix:\n")
        f.write(f"{cm}\n")
        f.write("Classification Report:\n")
        f.write(classification_report(y_test, y_pred))

    print(f"\nModel: {model_name}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Sensitivity (Recall): {sensitivity:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print(f"Latency: {latency:.4f} seconds")
    print("Confusion Matrix:")
    print(cm)
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

print("\nTraining Complete!")


Training Set Ratio: Counter({np.int64(0): 127028, np.int64(1): 798})
Test Set Ratio: Counter({np.int64(0): 31757, np.int64(1): 200})

Balanced Training Set Ratio: Counter({np.int64(0): 798, np.int64(1): 798})

Training Naive Bayes...

Model: Naive Bayes
F1 Score: 0.9684
Sensitivity (Recall): 0.6250
Specificity: 0.9509
Latency: 293.1960 seconds
Confusion Matrix:
[[30197  1560]
 [   75   125]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.95      0.97     31757
           1       0.07      0.62      0.13       200

    accuracy                           0.95     31957
   macro avg       0.54      0.79      0.55     31957
weighted avg       0.99      0.95      0.97     31957


Training Random Forest...

Model: Random Forest
F1 Score: 0.9714
Sensitivity (Recall): 0.9350
Specificity: 0.9539
Latency: 279.2515 seconds
Confusion Matrix:
[[30293  1464]
 [   13   187]]
Classification Report:
              precision    recall  f1-sco

/Users/folasewaabdulsalam/Seizure_Onset/venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [03:58:10] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



Model: XGBoost
F1 Score: 0.9723
Sensitivity (Recall): 0.9450
Specificity: 0.9555
Latency: 285.5241 seconds
Confusion Matrix:
[[30344  1413]
 [   11   189]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.96      0.98     31757
           1       0.12      0.94      0.21       200

    accuracy                           0.96     31957
   macro avg       0.56      0.95      0.59     31957
weighted avg       0.99      0.96      0.97     31957


Training Complete!


In [ ]:
from imblearn.under_sampling import RandomUnderSampler
# Load Data
features, labels = load_features_and_labels(feature_path, annotations_file)

# Preserve Original Seizure-to-Non-Seizure Ratio
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, stratify=labels, random_state=42
)

# Print Ratio
print("\nTraining Set Ratio:", Counter(y_train))
print("Test Set Ratio:", Counter(y_test))

# Normalize Features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Balance Training Data (Downsampling)
rus = RandomUnderSampler(random_state=42)
X_train_balanced, y_train_balanced = rus.fit_resample(X_train, y_train)

# Print Balanced Ratio
print("\nBalanced Training Set Ratio:", Counter(y_train_balanced))

# Define Models
models = {
    "Naive Bayes": GaussianNB(),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
}

# Train and Evaluate
for model_name, model in models.items():
    print(f"\nTraining {model_name}...")
    model.fit(X_train_balanced, y_train_balanced)
    y_pred = model.predict(X_test)

    # Metrics
    f1 = f1_score(y_test, y_pred, average="weighted")
    precision, recall, f1_scores, _ = precision_recall_fscore_support(y_test, y_pred, average="binary")
    cm = confusion_matrix(y_test, y_pred)

    # Sensitivity and Specificity
    tn, fp, fn, tp = cm.ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
        # Latency Calculation (Onset Delay)
    
    latency = np.mean([max(0, i - j) for i, j in zip(np.where(y_pred == 1)[0], np.where(y_test == 1)[0])]) if np.any(y_test == 1) else 0
    
    # Save Results
    with open(os.path.join(model_results_path, f"{model_name}_results4.txt"), "w") as f:
        f.write(f"Model: {model_name}\n")
        f.write(f"F1 Score: {f1:.4f}\n")
        f.write(f"Sensitivity (Recall): {sensitivity:.4f}\n")
        f.write(f"Specificity: {specificity:.4f}\n")
        f.write(f"Latency: {latency:.4f} seconds\n")
        f.write("Confusion Matrix:\n")
        f.write(f"{cm}\n")
        f.write("Classification Report:\n")
        f.write(classification_report(y_test, y_pred))

    print(f"\nModel: {model_name}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Sensitivity (Recall): {sensitivity:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print(f"Latency: {latency:.4f} seconds")
    print("Confusion Matrix:")
    print(cm)
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

print("\nTraining Complete!")


Training Set Ratio: Counter({np.int64(0): 127028, np.int64(1): 798})
Test Set Ratio: Counter({np.int64(0): 31757, np.int64(1): 200})

Balanced Training Set Ratio: Counter({np.int64(0): 798, np.int64(1): 798})

Training Naive Bayes...

Model: Naive Bayes
F1 Score: 0.9684
Sensitivity (Recall): 0.6250
Specificity: 0.9509
Latency: 0.0000 seconds
Confusion Matrix:
[[30197  1560]
 [   75   125]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.95      0.97     31757
           1       0.07      0.62      0.13       200

    accuracy                           0.95     31957
   macro avg       0.54      0.79      0.55     31957
weighted avg       0.99      0.95      0.97     31957


Training Random Forest...


/Users/folasewaabdulsalam/Seizure_Onset/venv/lib/python3.9/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/folasewaabdulsalam/Seizure_Onset/venv/lib/python3.9/site-packages/sklearn/base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
/Users/folasewaabdulsalam/Seizure_Onset/venv/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(



Model: Random Forest
F1 Score: 0.9705
Sensitivity (Recall): 0.9350
Specificity: 0.9524
Latency: 0.0000 seconds
Confusion Matrix:
[[30244  1513]
 [   13   187]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.95      0.98     31757
           1       0.11      0.94      0.20       200

    accuracy                           0.95     31957
   macro avg       0.55      0.94      0.59     31957
weighted avg       0.99      0.95      0.97     31957


Training XGBoost...


/Users/folasewaabdulsalam/Seizure_Onset/venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [04:00:00] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



Model: XGBoost
F1 Score: 0.9723
Sensitivity (Recall): 0.9450
Specificity: 0.9555
Latency: 0.0000 seconds
Confusion Matrix:
[[30344  1413]
 [   11   189]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.96      0.98     31757
           1       0.12      0.94      0.21       200

    accuracy                           0.96     31957
   macro avg       0.56      0.95      0.59     31957
weighted avg       0.99      0.96      0.97     31957


Training Complete!


In [103]:
#training 5
# Load Data
features, labels = load_features_and_labels(feature_path, annotations_file)

# Preserve Original Seizure-to-Non-Seizure Ratio
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, stratify=labels, random_state=42
)

# Print Ratio
print("\nTraining Set Ratio:", Counter(y_train))
print("Test Set Ratio:", Counter(y_test))

# Normalize Features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Balance Training Data
smote = SMOTE(random_state=42, k_neighbors=5)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Print Balanced Ratio
print("\nBalanced Training Set Ratio:", Counter(y_train_balanced))

# Define Models
models = {
    "Naive Bayes": GaussianNB(),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
}

# Train and Evaluate
for model_name, model in models.items():
    print(f"\nTraining {model_name}...")
    model.fit(X_train_balanced, y_train_balanced)
    y_pred = model.predict(X_test)

    # Metrics
    f1 = f1_score(y_test, y_pred, average="weighted")
    precision, recall, f1_scores, _ = precision_recall_fscore_support(y_test, y_pred, average="binary")
    cm = confusion_matrix(y_test, y_pred)

    # Sensitivity and Specificity
    tn, fp, fn, tp = cm.ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    # Latency Calculation (Onset Delay)
    latency = np.mean([max(0, i - j) for i, j in zip(np.where(y_pred == 1)[0], np.where(y_test == 1)[0])]) if np.any(y_test == 1) else 0

    # Save Results
    with open(os.path.join(model_results_path, f"{model_name}_results5.txt"), "w") as f:
        f.write(f"Model: {model_name}\n")
        f.write(f"F1 Score: {f1:.4f}\n")
        f.write(f"Sensitivity (Recall): {sensitivity:.4f}\n")
        f.write(f"Specificity: {specificity:.4f}\n")
        f.write(f"Latency: {latency:.4f} samples\n")
        f.write("Confusion Matrix:\n")
        f.write(f"{cm}\n")
        f.write("Classification Report:\n")
        f.write(classification_report(y_test, y_pred))

    print(f"\nModel: {model_name}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Sensitivity (Recall): {sensitivity:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print(f"Latency: {latency:.4f} samples")
    print("Confusion Matrix:")
    print(cm)
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

print("\nTraining Complete!")


Training Set Ratio: Counter({np.int64(0): 127028, np.int64(1): 798})
Test Set Ratio: Counter({np.int64(0): 31757, np.int64(1): 200})


/Users/folasewaabdulsalam/Seizure_Onset/venv/lib/python3.9/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



Balanced Training Set Ratio: Counter({np.int64(0): 127028, np.int64(1): 127028})

Training Naive Bayes...

Model: Naive Bayes
F1 Score: 0.9814
Sensitivity (Recall): 0.4800
Specificity: 0.9764
Latency: 0.4000 samples
Confusion Matrix:
[[31008   749]
 [  104    96]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     31757
           1       0.11      0.48      0.18       200

    accuracy                           0.97     31957
   macro avg       0.56      0.73      0.59     31957
weighted avg       0.99      0.97      0.98     31957


Training Random Forest...

Model: Random Forest
F1 Score: 0.9970
Sensitivity (Recall): 0.7200
Specificity: 0.9988
Latency: 1359.5519 samples
Confusion Matrix:
[[31718    39]
 [   56   144]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     31757
           1       0.79      0.72      0.75       200

    ac

/Users/folasewaabdulsalam/Seizure_Onset/venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [04:25:38] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



Model: XGBoost
F1 Score: 0.9975
Sensitivity (Recall): 0.8600
Specificity: 0.9983
Latency: 4.1150 samples
Confusion Matrix:
[[31703    54]
 [   28   172]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     31757
           1       0.76      0.86      0.81       200

    accuracy                           1.00     31957
   macro avg       0.88      0.93      0.90     31957
weighted avg       1.00      1.00      1.00     31957


Training Complete!


In [104]:
#training 6 with downsample
# Load Data
features, labels = load_features_and_labels(feature_path, annotations_file)

# Preserve Original Seizure-to-Non-Seizure Ratio
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, stratify=labels, random_state=42
)

# Print Ratio
print("\nTraining Set Ratio:", Counter(y_train))
print("Test Set Ratio:", Counter(y_test))

# Normalize Features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Balance Training Data with Downsampling
rus = RandomUnderSampler(random_state=42)
X_train_balanced, y_train_balanced = rus.fit_resample(X_train, y_train)

# Print Balanced Ratio
print("\nBalanced Training Set Ratio:", Counter(y_train_balanced))

# Define Models
models = {
    "Naive Bayes": GaussianNB(),
    "Random Forest": RandomForestClassifier(n_estimators=200, max_depth=15, random_state=42),
    "XGBoost": XGBClassifier(n_estimators=150, max_depth=10, learning_rate=0.05, 
                             subsample=0.8, colsample_bytree=0.8, eval_metric='logloss', random_state=42)
}

# Train and Evaluate
for model_name, model in models.items():
    print(f"\nTraining {model_name}...")
    model.fit(X_train_balanced, y_train_balanced)
    y_pred = model.predict(X_test)

    # Metrics
    f1 = f1_score(y_test, y_pred, average="weighted")
    precision, recall, f1_scores, _ = precision_recall_fscore_support(y_test, y_pred, average="binary")
    cm = confusion_matrix(y_test, y_pred)

    # Sensitivity and Specificity
    tn, fp, fn, tp = cm.ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    # Latency Calculation (Onset Delay)
    latency = np.mean([max(0, i - j) for i, j in zip(np.where(y_pred == 1)[0], np.where(y_test == 1)[0])]) if np.any(y_test == 1) else 0

    # Save Results
    with open(os.path.join(model_results_path, f"{model_name}_results6.txt"), "w") as f:
        f.write(f"Model: {model_name}\n")
        f.write(f"F1 Score: {f1:.4f}\n")
        f.write(f"Sensitivity (Recall): {sensitivity:.4f}\n")
        f.write(f"Specificity: {specificity:.4f}\n")
        f.write(f"Latency: {latency:.4f} samples\n")
        f.write("Confusion Matrix:\n")
        f.write(f"{cm}\n")
        f.write("Classification Report:\n")
        f.write(classification_report(y_test, y_pred))

    print(f"\nModel: {model_name}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Sensitivity (Recall): {sensitivity:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print(f"Latency: {latency:.4f} samples")
    print("Confusion Matrix:")
    print(cm)
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

print("\nTraining Complete!")


Training Set Ratio: Counter({np.int64(0): 127028, np.int64(1): 798})
Test Set Ratio: Counter({np.int64(0): 31757, np.int64(1): 200})

Balanced Training Set Ratio: Counter({np.int64(0): 798, np.int64(1): 798})

Training Naive Bayes...


/Users/folasewaabdulsalam/Seizure_Onset/venv/lib/python3.9/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/folasewaabdulsalam/Seizure_Onset/venv/lib/python3.9/site-packages/sklearn/base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
/Users/folasewaabdulsalam/Seizure_Onset/venv/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(



Model: Naive Bayes
F1 Score: 0.9684
Sensitivity (Recall): 0.6250
Specificity: 0.9509
Latency: 0.0000 samples
Confusion Matrix:
[[30197  1560]
 [   75   125]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.95      0.97     31757
           1       0.07      0.62      0.13       200

    accuracy                           0.95     31957
   macro avg       0.54      0.79      0.55     31957
weighted avg       0.99      0.95      0.97     31957


Training Random Forest...

Model: Random Forest
F1 Score: 0.9706
Sensitivity (Recall): 0.9350
Specificity: 0.9525
Latency: 0.0000 samples
Confusion Matrix:
[[30248  1509]
 [   13   187]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.95      0.98     31757
           1       0.11      0.94      0.20       200

    accuracy                           0.95     31957
   macro avg       0.55      0.94      0.59     31957
weighted

In [108]:
# Training 7 - Corrected Latency and Balanced Downsampling

# Load Features and Labels
features, labels = load_features_and_labels(feature_path, annotations_file)

print("\nTotal Features:", features.shape)
print("Total Labels:", labels.shape)
print("Initial Label Distribution:", Counter(labels))

# Stratified Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, stratify=labels, random_state=42
)

print("\nTraining Set Shape:", X_train.shape)
print("Test Set Shape:", X_test.shape)
print("Training Labels Distribution:", Counter(y_train))
print("Test Labels Distribution:", Counter(y_test))

# Downsampling the Majority Class
rus = RandomUnderSampler(random_state=42, sampling_strategy={0: 5000, 1: 798})
X_train_balanced, y_train_balanced = rus.fit_resample(X_train, y_train)

print("\nBalanced Training Labels Distribution:", Counter(y_train_balanced))

# Standardize Features
scaler = StandardScaler()
X_train_balanced = scaler.fit_transform(X_train_balanced)
X_test = scaler.transform(X_test)

# Model Definitions
models = {
    "Naive Bayes": GaussianNB(),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
}

# Helper function to calculate latency, sensitivity, and specificity
def evaluate_onset_detection(y_true, y_pred, window_size=2, fs=256):
    """
    Calculate latency, sensitivity, and specificity.
    """
    # Convert window indices to time
    onset_times_true = np.where(y_true == 1)[0]
    onset_times_pred = np.where(y_pred == 1)[0]

    # Calculate latency
    latencies = []
    for true_onset in onset_times_true:
        # Find the first correct prediction after the actual onset
        valid_preds = onset_times_pred[onset_times_pred >= true_onset]
        if len(valid_preds) > 0:
            latency = (valid_preds[0] - true_onset) * (window_size / fs)
            latencies.append(latency)

    # Average latency across all seizure events
    avg_latency = np.mean(latencies) if len(latencies) > 0 else float('inf')

    # Calculate sensitivity (true positive rate)
    sensitivity = len(latencies) / len(onset_times_true) if len(onset_times_true) > 0 else 0

    # Calculate specificity (true negative rate)
    non_seizure_true = np.sum(y_true == 0)
    non_seizure_pred_correct = np.sum((y_pred == 0) & (y_true == 0))
    specificity = non_seizure_pred_correct / non_seizure_true if non_seizure_true > 0 else 0

    return avg_latency, sensitivity, specificity

# Train and Evaluate Models
for model_name, model in models.items():
    print(f"\nTraining {model_name}...")
    model.fit(X_train_balanced, y_train_balanced)
    y_pred = model.predict(X_test)

    # Metrics
    f1 = f1_score(y_test, y_pred, average="weighted")
    precision, recall, f1_scores, _ = precision_recall_fscore_support(y_test, y_pred, average="binary")
    cm = confusion_matrix(y_test, y_pred)

    # Sensitivity, Specificity, and Latency
    latency, sensitivity, specificity = evaluate_onset_detection(y_test, y_pred)
    
    # Save Results
    results_file = os.path.join(model_results_path, f"{model_name}_results7.txt")
    with open(results_file, "w") as f:
        f.write(f"Model: {model_name}\n")
        f.write(f"F1 Score: {f1:.4f}\n")
        f.write(f"Sensitivity (Recall): {sensitivity:.4f}\n")
        f.write(f"Specificity: {specificity:.4f}\n")
        f.write(f"Latency: {latency:.4f} seconds\n")
        f.write("Confusion Matrix:\n")
        f.write(f"{cm}\n")
        f.write("Classification Report:\n")
        f.write(classification_report(y_test, y_pred))

    print(f"\nModel: {model_name}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Sensitivity (Recall): {sensitivity:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print(f"Latency: {latency:.4f} seconds")
    print("Confusion Matrix:")
    print(cm)
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

print("\nModel training and evaluation complete!")



Total Features: (159783, 144)
Total Labels: (159783,)
Initial Label Distribution: Counter({np.int64(0): 158785, np.int64(1): 998})

Training Set Shape: (127826, 144)
Test Set Shape: (31957, 144)
Training Labels Distribution: Counter({np.int64(0): 127028, np.int64(1): 798})
Test Labels Distribution: Counter({np.int64(0): 31757, np.int64(1): 200})

Balanced Training Labels Distribution: Counter({np.int64(0): 5000, np.int64(1): 798})

Training Naive Bayes...


/Users/folasewaabdulsalam/Seizure_Onset/venv/lib/python3.9/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/folasewaabdulsalam/Seizure_Onset/venv/lib/python3.9/site-packages/sklearn/base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
/Users/folasewaabdulsalam/Seizure_Onset/venv/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(



Model: Naive Bayes
F1 Score: 0.9808
Sensitivity (Recall): 1.0000
Specificity: 0.9752
Latency: 0.1348 seconds
Confusion Matrix:
[[30971   786]
 [  102    98]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     31757
           1       0.11      0.49      0.18       200

    accuracy                           0.97     31957
   macro avg       0.55      0.73      0.58     31957
weighted avg       0.99      0.97      0.98     31957


Training Random Forest...

Model: Random Forest
F1 Score: 0.9917
Sensitivity (Recall): 1.0000
Specificity: 0.9908
Latency: 0.1063 seconds
Confusion Matrix:
[[31466   291]
 [   36   164]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     31757
           1       0.36      0.82      0.50       200

    accuracy                           0.99     31957
   macro avg       0.68      0.91      0.75     31957
weighted

/Users/folasewaabdulsalam/Seizure_Onset/venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [04:59:42] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



Model: XGBoost
F1 Score: 0.9927
Sensitivity (Recall): 1.0000
Specificity: 0.9916
Latency: 0.0654 seconds
Confusion Matrix:
[[31489   268]
 [   20   180]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     31757
           1       0.40      0.90      0.56       200

    accuracy                           0.99     31957
   macro avg       0.70      0.95      0.78     31957
weighted avg       1.00      0.99      0.99     31957


Model training and evaluation complete!


In [109]:
#training 8 lstm
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support
from sklearn.preprocessing import StandardScaler
from collections import Counter
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

# Load Features and Labels
features, labels = load_features_and_labels(feature_path, annotations_file)

print("\nTotal Features:", features.shape)
print("Total Labels:", labels.shape)
print("Initial Label Distribution:", Counter(labels))

# Stratified Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, stratify=labels, random_state=42
)

print("\nTraining Set Shape:", X_train.shape)
print("Test Set Shape:", X_test.shape)
print("Training Labels Distribution:", Counter(y_train))
print("Test Labels Distribution:", Counter(y_test))

# Standardize Features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape for LSTM (samples, time steps, features)
X_train = np.expand_dims(X_train, axis=1)
X_test = np.expand_dims(X_test, axis=1)

# LSTM Model Definition
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(1, X_train.shape[2])),
    Dropout(0.3),
    LSTM(64),
    Dropout(0.3),
    Dense(32, activation="relu"),
    Dense(1, activation="sigmoid")
])

# Compile the Model
model.compile(optimizer=Adam(learning_rate=0.001), loss="binary_crossentropy", metrics=["accuracy"])

# Early Stopping
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

# Train the Model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=64,
    callbacks=[early_stopping]
)

# Evaluate the Model
y_pred = (model.predict(X_test) > 0.5).astype("int32").flatten()

# Metrics
precision, recall, f1_scores, _ = precision_recall_fscore_support(y_test, y_pred, average="binary")
cm = confusion_matrix(y_test, y_pred)

# Sensitivity and Specificity
tn, fp, fn, tp = cm.ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

# Latency Calculation (Samples to Seconds)
latency = np.mean([max(0, i - j) for i, j in zip(np.where(y_pred == 1)[0], np.where(y_test == 1)[0])]) * 2

# Save Results
model_results_path = "/Users/folasewaabdulsalam/Seizure_Onset/model_results"
os.makedirs(model_results_path, exist_ok=True)
results_file = os.path.join(model_results_path, "LSTM_results.txt")
with open(results_file, "w") as f:
    f.write(f"F1 Score: {f1_scores:.4f}\n")
    f.write(f"Sensitivity (Recall): {sensitivity:.4f}\n")
    f.write(f"Specificity: {specificity:.4f}\n")
    f.write(f"Latency: {latency:.4f} seconds\n")
    f.write("Confusion Matrix:\n")
    f.write(f"{cm}\n")
    f.write("Classification Report:\n")
    f.write(classification_report(y_test, y_pred))

print(f"\nF1 Score: {f1_scores:.4f}")
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"Latency: {latency:.4f} seconds")
print("Confusion Matrix:")
print(cm)
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("\nLSTM Training and Evaluation Complete!")


/Users/folasewaabdulsalam/Seizure_Onset/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(



Total Features: (159783, 144)
Total Labels: (159783,)
Initial Label Distribution: Counter({np.int64(0): 158785, np.int64(1): 998})

Training Set Shape: (127826, 144)
Test Set Shape: (31957, 144)
Training Labels Distribution: Counter({np.int64(0): 127028, np.int64(1): 798})
Test Labels Distribution: Counter({np.int64(0): 31757, np.int64(1): 200})
Epoch 1/30


/Users/folasewaabdulsalam/Seizure_Onset/venv/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1998/1998 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.9937 - loss: 0.0686 - val_accuracy: 0.9966 - val_loss: 0.0114
Epoch 2/30
1998/1998 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9966 - loss: 0.0127 - val_accuracy: 0.9973 - val_loss: 0.0094
Epoch 3/30
1998/1998 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9973 - loss: 0.0109 - val_accuracy: 0.9975 - val_loss: 0.0089
Epoch 4/30
1998/1998 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9977 - loss: 0.0082 - val_accuracy: 0.9978 - val_loss: 0.0082
Epoch 5/30
1998/1998 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9981 - loss: 0.0076 - val_accuracy: 0.9977 - val_loss: 0.0078
Epoch 6/30
1998/1998 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.9980 - loss: 0.0075 - val_accuracy: 0.9975 - val_loss: 0.0088
Epoch 7/30
1998/1998 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.9982 - loss: 0.0069 - val_accuracy: 0.9976 - val_loss: 0.0083
Epoch 8/30
1998/1998 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9985 - loss: 0.0061 - val_accurac

In [110]:
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, precision_recall_fscore_support, confusion_matrix, classification_report
from imblearn.under_sampling import RandomUnderSampler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
import numpy as np
import os


# Load Features and Labels
features, labels = load_features_and_labels(feature_path, annotations_file)

print("\nTotal Features:", features.shape)
print("Total Labels:", labels.shape)
print("Initial Label Distribution:", Counter(labels))

# Stratified Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, stratify=labels, random_state=42
)

print("\nTraining Set Shape:", X_train.shape)
print("Test Set Shape:", X_test.shape)
print("Training Labels Distribution:", Counter(y_train))
print("Test Labels Distribution:", Counter(y_test))

# Downsampling the Majority Class
rus = RandomUnderSampler(random_state=42)
X_train_balanced, y_train_balanced = rus.fit_resample(X_train, y_train)

print("\nBalanced Training Labels Distribution:", Counter(y_train_balanced))

# Standardize Features
scaler = StandardScaler()
X_train_balanced = scaler.fit_transform(X_train_balanced)
X_test = scaler.transform(X_test)

# Reshape for LSTM (samples, timesteps, features)
X_train_balanced = X_train_balanced.reshape(-1, 1, X_train_balanced.shape[1])
X_test = X_test.reshape(-1, 1, X_test.shape[1])

# LSTM Model Definition
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(1, 144)),
    BatchNormalization(),
    Dropout(0.5),
    LSTM(64),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Train the LSTM Model
model.fit(X_train_balanced, y_train_balanced, epochs=20, batch_size=32, validation_split=0.2)

# Predict on Test Set
y_pred = (model.predict(X_test) > 0.5).astype(int).flatten()

# Metrics
f1 = f1_score(y_test, y_pred, average="weighted")
precision, recall, f1_scores, _ = precision_recall_fscore_support(y_test, y_pred, average="binary")
cm = confusion_matrix(y_test, y_pred)

# Sensitivity, Specificity, and Latency
tn, fp, fn, tp = cm.ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

# Overlapping Window Latency Calculation
window_size = 2  # seconds
fs = 256  # sampling rate
latencies = []
onset_times_true = np.where(y_test == 1)[0]
onset_times_pred = np.where(y_pred == 1)[0]

for true_onset in onset_times_true:
    valid_preds = onset_times_pred[onset_times_pred >= true_onset]
    if len(valid_preds) > 0:
        latency = (valid_preds[0] - true_onset) * (window_size / 2)  # account for 50% overlap
        latencies.append(latency)

avg_latency = np.mean(latencies) if len(latencies) > 0 else float('inf')

# Save Results
results_file = os.path.join(model_results_path, "LSTM_results2.txt")
with open(results_file, "w") as f:
    f.write(f"F1 Score: {f1:.4f}\n")
    f.write(f"Sensitivity (Recall): {sensitivity:.4f}\n")
    f.write(f"Specificity: {specificity:.4f}\n")
    f.write(f"Latency: {avg_latency:.4f} seconds\n")
    f.write("Confusion Matrix:\n")
    f.write(f"{cm}\n")
    f.write("Classification Report:\n")
    f.write(classification_report(y_test, y_pred))

print("\nLSTM Model Results")
print(f"F1 Score: {f1:.4f}")
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"Latency: {avg_latency:.4f} seconds")
print("Confusion Matrix:")
print(cm)
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("\nLSTM training and evaluation complete!")



Total Features: (159783, 144)
Total Labels: (159783,)
Initial Label Distribution: Counter({np.int64(0): 158785, np.int64(1): 998})

Training Set Shape: (127826, 144)
Test Set Shape: (31957, 144)
Training Labels Distribution: Counter({np.int64(0): 127028, np.int64(1): 798})
Test Labels Distribution: Counter({np.int64(0): 31757, np.int64(1): 200})

Balanced Training Labels Distribution: Counter({np.int64(0): 798, np.int64(1): 798})


/Users/folasewaabdulsalam/Seizure_Onset/venv/lib/python3.9/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/folasewaabdulsalam/Seizure_Onset/venv/lib/python3.9/site-packages/sklearn/base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
/Users/folasewaabdulsalam/Seizure_Onset/venv/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(
/Users/folasewaabdulsalam/Seizure_Onset/venv/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Inpu

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 1, 64)          │        53,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1, 64)          │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 87,105 (340.25 KB)

 Trainable params: 86,849 (339.25 KB)

 Non-trainable params: 256 (1.00 KB)

Epoch 1/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7326 - loss: 0.6275 - val_accuracy: 0.4187 - val_loss: 0.7127
Epoch 2/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8643 - loss: 0.3877 - val_accuracy: 0.5469 - val_loss: 0.6463
Epoch 3/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8652 - loss: 0.3711 - val_accuracy: 0.6406 - val_loss: 0.5881
Epoch 4/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8844 - loss: 0.3111 - val_accuracy: 0.7000 - val_loss: 0.5228
Epoch 5/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8693 - loss: 0.3201 - val_accuracy: 0.8031 - val_loss: 0.4521
Epoch 6/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9071 - loss: 0.2732 - val_accuracy: 0.8250 - val_loss: 0.3968
Epoch 7/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9049 - loss: 0.2452 - val_accuracy: 0.8344 - val_loss: 0.3502
Epoch 8/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8731 - loss: 0.3157 - val_accuracy: 0.8375 - val_loss: